In [8]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [9]:
#grab data

df_look_up = pd.read_csv("../../data/raw/EXTR_LookUp.csv")

df_parcel = pd.read_csv("../../data/raw/EXTR_Parcel.csv")
df_res_building = pd.read_csv("../../data/raw/EXTR_ResBldg.csv")
df_rp_sale = pd.read_csv("../../data/raw/EXTR_RPSale.csv")

In [10]:
# df_look_up.head()
# df_parcel.head()
# df_res_building.head()
# df_rp_sale.head()

In [11]:
# df_look_up.columns
# df_parcel.columns
# df_res_building.columns
# df_rp_sale.columns

In [12]:
import re

In [13]:
#let's rename some of the column names
def name_fixer(to_fix):
    to_fix = to_fix.strip()
    to_fix = re.sub(r"(\w)([A-Z])", r"\1_\2", to_fix)
    return to_fix.lower()

In [14]:
#Remove any trailing whitespace in data
def whitespace_fixer(elements):
    elements = elements.applymap(lambda x : x.strip() if type(x) == type("str") else x)
    return elements

In [15]:
def id_glue(first, second):
    return int(str(first) + str(second))

In [16]:
test = "TestingThisTThing"
#print(name_fixer(test))

In [17]:
df_parcel = df_parcel.rename(columns = lambda x: name_fixer(x))
df_res_building = df_res_building.rename(columns = lambda x: name_fixer(x))
df_rp_sale = df_rp_sale.rename(columns = lambda x: name_fixer(x))

In [18]:
# df_look_up.columns
# df_parcel.columns
#df_res_building.columns
#df_rp_sale.columns

In [19]:
# fix df_parcel
df_parcel = df_parcel.rename(columns = {
    'd_nr_lease' : 'dnr_lease',
    'h_bu_as_if_vacant' : 'hbu_as_if_vacant'
})
# df_parcel.columns

In [20]:
df_rp_sale = df_rp_sale.rename(columns = {
    'a_fforest_land' : 'af_forest_land',
    'a_fcurrent_use_land' : 'af_current_use_land',
    'a_fnon_profit_use' : 'af_non_profit_use',
    'a_fhistoric_property' : 'af_historic_property'
})
# df_rp_sale.columns

In [21]:
# df_look_up.info()
#df_parcel.info()
# df_res_building.info()
# df_rp_sale.info()

In [22]:
parcel_cols = ['major', 'minor', 'prop_type', 'sq_ft_lot', 'water_system', 
               'sewer_system', 'inadequate_parking', 'pcnt_unusable', 'unbuildable', 
               'nbr_bldg_sites', 'contamination', 'water_problems']

In [23]:
#not wanted columns
res_building_cols = ['bldg_nbr','address','fraction','direction_prefix', 'street_name',
       'street_type', 'direction_suffix', 'zip_code', 'bldg_grade_var']

In [24]:
rp_sale_cols = ["sale_price", "major", "minor", "document_date"]

In [25]:
#clean df_parcel
df_parcel_c = df_parcel[parcel_cols].copy()

In [26]:
#make id_number
for row in [df_parcel_c]:
    row['id_number'] = row['major'].astype(str) + row['minor'].astype(str)
    row.drop(columns = ['major', 'minor'], inplace = True)

In [27]:
#df_parcel_c.head()
#df_parcel_c.info()
#df_parcel_c['id_number'].value_counts()

In [28]:
#clean df_res_building
df_res_building_c = df_res_building.drop(columns = res_building_cols)

In [29]:
#df_res_building_c.head()
#res_building_cols
#df_res_building.info()

In [30]:
#fix types for columns
res_building_int = ['nbr_living_units', 'bedrooms'] + \
              [c for c in df_res_building_c.columns if 'sqft' in c.lower() or 'count' in c.lower()] + \
              ['brick_stone', 'pcnt_complete', 'pcnt_net_condition', 'yr_built', 'yr_renovated', 'addnl_cost']
res_building_fl = ['stories']

In [31]:
for c in res_building_int:
    df_res_building_c[c] = df_res_building_c[c].astype(int)
for c in res_building_fl:
    df_res_building[c] = df_res_building_c[c].astype(float)

In [32]:
#make id_number
for row in [df_res_building_c]:
    row['id_number'] = row['major'].astype(str) + row['minor'].astype(str)
    row.drop(columns = ['major', 'minor'], inplace = True)

In [33]:
#df_res_building_c.head()

In [34]:
#clean df_rp_sale
df_rp_sale_c = whitespace_fixer(df_rp_sale[rp_sale_cols].copy())

In [35]:
from datetime import datetime

In [36]:
#change document_date to year
df_rp_sale_c['document_date'] = pd.to_datetime(df_rp_sale_c.document_date, infer_datetime_format = True)
df_rp_sale_c['year'] = df_rp_sale_c['document_date'].dt.year

In [37]:
#drop document_date
df_rp_sale_c.drop(columns = ['document_date'], inplace = True)

In [38]:
#df_rp_sale_c.head()
#df_rp_sale_c.columns

In [39]:
#only 2019 data
df_rp_sale_c = df_rp_sale_c[df_rp_sale_c.year == 2019]

In [40]:
#change sale_price type to int
df_rp_sale_c.sale_price = df_rp_sale_c.sale_price.astype(int)

In [41]:
#drop rows with sale price of less than 0
df_rp_sale_c = df_rp_sale_c[df_rp_sale_c.sale_price > 0]

In [42]:
#make id_number
for row in [df_rp_sale_c]:
    row['id_number'] = row['major'].astype(str) + row['minor'].astype(str)
    row.drop(columns = ['major', 'minor'], inplace = True)

In [43]:
df_rp_sale_c.head()

,sale_price,year,id_number
2,192000,2019,919715200
8,185000,2019,894444200
9,560000,2019,213043120
10,435000,2019,940652630
19,7600000,2019,1525049008


In [44]:
#Utilizing df_look_up
#df_look_up.head()

In [45]:
df_look_up = whitespace_fixer(df_look_up)

In [46]:
#df_look_up[df_look_up['LUType'] == 1]
df_look_up[(df_look_up['LUType'] == 1) & (df_look_up['LUItem'] == 1)]

,LUType,LUItem,LUDescription
0,1,1,LAND ONLY


In [47]:
#retrieve dataframe for requested types and items
# def lookup_(t):
#     return df_look_up[df_look_up['LUType'] == t]

def lookup_item(t, i):
    temp_df = df_look_up[(df_look_up['LUType'] == t) & (df_look_up['LUItem'] == i)]
    if (len(temp_df) == 1):
        return temp_df['LUDescription'].iloc[0]
    return i

# def lookup_item_test(t,i):
#     b = 0
#     return (df_look_up['LUType'] == t) & (df_look_up['LUItem'] == i)

In [48]:
#df_look_up.head()
#df_res_building_c.head()
#df_parcel_c.head()
#df_rp_sale_c.head()

In [49]:
df_res_building_c = df_res_building_c.drop_duplicates(subset = ['id_number'], keep = 'last')
df_parcel_c = df_parcel_c.drop_duplicates(subset = ['id_number'], keep = 'last')
df_rp_sale_c = df_rp_sale_c.drop_duplicates(subset = ['id_number'], keep = 'last')

In [50]:
df_res_building_c['id_number'].value_counts()

785351160     1
813210210     1
675200350     1
48600125      1
290927210     1
             ..
1022049109    1
769460253     1
124059060     1
321124250     1
3356406210    1
Name: id_number, Length: 179475, dtype: int64

In [51]:
df_res_building_c.head()

,nbr_living_units,building_number,stories,bldg_grade,sq_ft1st_floor,sq_ft_half_floor,sq_ft2nd_floor,sq_ft_upper_floor,sq_ft_unfin_full,sq_ft_unfin_half,...,fp_freestanding,fp_additional,yr_built,yr_renovated,pcnt_complete,obsolescence,pcnt_net_condition,condition,addnl_cost,id_number
0,1,27719,2.0,11,1970,0,2130,0,0,0,...,0,0,2001,0,0,0,0,3,0,9800720
1,1,2829,2.0,10,1610,0,1400,0,0,0,...,0,0,2004,0,0,0,0,3,0,9802140
2,1,1715,2.0,10,2520,0,2560,0,0,0,...,0,0,2017,0,0,0,0,3,0,983020
3,1,1861,2.0,10,2210,0,1860,0,0,0,...,0,0,2013,0,0,0,0,3,0,9830160
4,1,35410,2.0,7,910,0,700,0,0,0,...,0,0,1994,0,0,0,0,4,0,10050180


In [52]:
df_first = df_res_building_c.merge(df_parcel_c, on = 'id_number', how = 'inner')

In [53]:
df = df_first.merge(df_rp_sale_c, on = 'id_number', how = 'inner')

In [54]:
df.head()

,nbr_living_units,building_number,stories,bldg_grade,sq_ft1st_floor,sq_ft_half_floor,sq_ft2nd_floor,sq_ft_upper_floor,sq_ft_unfin_full,sq_ft_unfin_half,...,water_system,sewer_system,inadequate_parking,pcnt_unusable,unbuildable,nbr_bldg_sites,contamination,water_problems,sale_price,year
0,1,2435,1.0,7,1480,0,0,0,0,0,...,2,2,2,0,False,0,0,N,311000,2019
1,1,12254,1.0,5,550,0,0,0,0,0,...,2,1,2,0,False,0,0,N,188500,2019
2,1,1602,2.0,9,660,0,595,0,0,0,...,2,2,0,0,False,0,0,N,730000,2019
3,1,3410,2.0,9,1602,0,1991,0,0,0,...,2,2,2,0,False,0,0,N,888679,2019
4,1,4512,1.0,8,2290,0,0,0,0,0,...,2,2,2,0,False,0,0,N,800000,2019


In [55]:
#df.isna().sum().sort_values(ascending=False)[:3]

In [56]:
# #columns that have a dictionary refrence
# nbr_living_units         none
# building_number          none
# stories                  none
# bldg_grade               code: 82
# sq_ft1st_floor           none
# sq_ft_half_floor         none
# sq_ft2nd_floor           none
# sq_ft_upper_floor        none
# sq_ft_unfin_full         none
# sq_ft_unfin_half         none
# sq_ft_tot_living         none
# sq_ft_tot_basement       none
# sq_ft_fin_basement       none
# fin_basement_grade       code: 82
# sq_ft_garage_basement    none
# sq_ft_garage_attached    none
# daylight_basement        none
# sq_ft_open_porch         none
# sq_ft_enclosed_porch     none
# sq_ft_deck               none
# heat_system              code: 108
# heat_source              code: 84
# brick_stone              none
# view_utilization         none
# bedrooms                 none
# bath_half_count          none
# bath3qtr_count           none
# bath_full_count          none
# fp_single_story          none
# fp_multi_story           none
# fp_freestanding          none
# fp_additional            none
# yr_built                 none
# yr_renovated             none
# pcnt_complete            none
# obsolescence             none
# pcnt_net_condition       none
# condition                code: 83
# addnl_cost               none
# id_number                none
# prop_type                code: 1
# sq_ft_lot                none
# water_system             code: 56
# sewer_system             code: 57
# inadequate_parking       code: 92
# pcnt_unusable            none
# unbuildable              none
# nbr_bldg_sites           none
# contamination            code: 93
# water_problems           none
# sale_price               none
# year                     none

In [57]:
def column_filler(column_name, type_):
    new_column_name = column_name + '_val'
    df_refrence = lookup_(type_)
    for row in [df]:
        row[new_column_name] = lookup_item(type_, row[column_name])['LUDescription'][1]
        #row[new_column_name] = row[column_name]
        row.drop(columns = ['column_name'], inplace = True)

In [58]:
columns_to_fix = {'bldg_grade' : 82, 
                  'fin_basement_grade' : 82, 
                  'heat_system' : 108, 
                  'heat_source' : 84,
                  'condition' : 83,
                  'prop_type' : 1,
                  'water_system' : 56,
                  'sewer_system' : 57,
                  'inadequate_parking' : 92,
                  'contamination' : 93
}

In [68]:
for column_ in columns_to_fix:
    code = columns_to_fix[column_]
    df[column_] = df[column_].apply(lambda x: lookup_item(code, x))

C:\Users\chris\anaconda3\envs\learn-env\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [66]:
#df['bldg_grade'] = df['bldg_grade'].apply(lambda x: lookup_item(82, x))

In [77]:
df['daylight_basement'].value_counts()

N    11528
      9802
Y     7307
Name: daylight_basement, dtype: int64

In [78]:
df['view_utilization'].value_counts()

     15016
N    12997
Y      624
Name: view_utilization, dtype: int64

In [75]:
df['water_problems'].value_counts()

N    28549
Y       88
Name: water_problems, dtype: int64

In [79]:
#last step, fix the y and n
df['daylight_basement'] = df['daylight_basement'].apply(lambda x: x.upper())
df['view_utilization'] = df['view_utilization'].apply(lambda x: x.upper())
#fixed

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28637 entries, 0 to 28636
Data columns (total 52 columns):
nbr_living_units         28637 non-null int32
building_number          28637 non-null object
stories                  28637 non-null float64
bldg_grade               28637 non-null object
sq_ft1st_floor           28637 non-null int64
sq_ft_half_floor         28637 non-null int64
sq_ft2nd_floor           28637 non-null int64
sq_ft_upper_floor        28637 non-null int64
sq_ft_unfin_full         28637 non-null int64
sq_ft_unfin_half         28637 non-null int64
sq_ft_tot_living         28637 non-null int64
sq_ft_tot_basement       28637 non-null int64
sq_ft_fin_basement       28637 non-null int64
fin_basement_grade       28637 non-null object
sq_ft_garage_basement    28637 non-null int64
sq_ft_garage_attached    28637 non-null int64
daylight_basement        28637 non-null object
sq_ft_open_porch         28637 non-null int64
sq_ft_enclosed_porch     28637 non-null int64
sq_ft_deck 

In [69]:
df['heat_system']

0        Forced Air
1        Floor-Wall
2        Floor-Wall
3        Forced Air
4         Hot Water
            ...    
28632    Forced Air
28633     Heat Pump
28634     Heat Pump
28635    Forced Air
28636    Forced Air
Name: heat_system, Length: 28637, dtype: object

In [92]:
df.to_csv('../../data/processed/kc_clean_data.csv')

In [63]:
#lookup_item(1,21)

'Food and kindred products'

In [242]:
#df.loc[50]['sale_price']

505000

AttributeError: 'NoneType' object has no attribute 'axes'

In [93]:
df_3 = df_2[df_2.PropertyType == 'Household, single family units'].copy()

RecursionError: maximum recursion depth exceeded